In [ ]:
#安装python开发web程序的依赖
!conda install flask

In [ ]:
from flask import Flask
# 创建一个网页
app = Flask(__name__)

@app.route("/aaaa")
def gaga():
    return '<h1>静夜思</h1> <h2 style="color:blue" >床前明月光</h2> <h2>疑是地上霜</h2>'

@app.route("/bbbb")
def haha():
    return "haha"

if __name__ == '__main__':
    app.run(port=80)

## websoket 

In [ ]:
##  socketio eventlet socket通讯需要的依赖
## conda install -c conda-forge python-socketio
## conda install -c conda-forge eventlet

In [ ]:
import eventlet
import socketio
from flask import Flask

sio = socketio.Server()  ## 创建socketio的服务器
app = Flask(__name__)

@sio.on('connect')
def connect(sid,env):
    print("websocket链接建立了")
    sio.emit("steer",data={
        'steering_angle':'0',
        'throttle':'1'
    })

if __name__=='__main__':
    # 把web服务器和socketio绑定在一起。
    app = socketio.Middleware(sio,app)
    ## 监听socket的事件了
    eventlet.wsgi.server(eventlet.listen(('',4567)),app)


In [ ]:
## 包装方法
## 图片预处理的逻辑，接受参数图片的路径
def img_preprocess(src_image):
   
    img = src_image[60:135,:,:]
    img = cv2.GaussianBlur(img,(3,3),1)
    img = cv2.resize(img,(200,66))
    img = cv2.cvtColor(img,cv2.COLOR_RGB2YUV)
    return img

import tensorflow as tf
save_model = tf.keras.models.load_model('hahaha.h5')
## 从模拟器汽车的摄像头里面读到数据， 基于图像输入，使用cnn训练好的模型，预测方向盘的角度。

import eventlet
import socketio
from flask import Flask
from io import BytesIO
from PIL import Image
import cv2
import base64
import numpy as np

sio = socketio.Server()  ## 创建socketio的服务器
app = Flask(__name__)

speed_limit = 8

@sio.on('connect')
def connect(sid,env):
    print("websocket链接建立了")
#     sio.emit("steer",data={
#         'steering_angle':'0',
#         'throttle':'0.0'
#     })


def send_control(angle,throttle):
    sio.emit("steer",data={
        'steering_angle':angle.__str__(),
        'throttle':throttle.__str__()
    })

    
@sio.on('telemetry')
def telemetry(sid,data):
    image = Image.open(BytesIO(base64.b64decode(data['image'])))
    image = np.asarray(image)
    cv2.imshow("info",image)
    cv2.waitKey(10)
    
    #预测
    image = img_preprocess(image)
    image = image / 255.0
    angel = save_model.predict(image.reshape(1,66,200,3))[0][0]
    
    ##当前的车速
    speed = float(data['speed'])
    throttle = 1.0 - speed / speed_limit
    
    send_control(angel,throttle)
    
if __name__=='__main__':
    # 把web服务器和socketio绑定在一起。
    app = socketio.Middleware(sio,app)
    ## 监听socket的事件了
    eventlet.wsgi.server(eventlet.listen(('',4567)),app)
